Business Understanding

The boss man wants low risk aircraft. 

# Data Understanding

In [198]:
import pandas as pd
import numpy as np

In [210]:
ad = pd.read_csv('Aviation_Data.csv') #named dataset "ad" for simplicity

C:\Users\joshg\anaconda3\envs\learn-env\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (6,7,28) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [219]:
ad.info() #see what the dataset looks like


<class 'pandas.core.frame.DataFrame'>
Int64Index: 28813 entries, 5 to 90345
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Accident.Number         28813 non-null  object 
 1   Aircraft.Category       28813 non-null  object 
 2   Amateur.Built           28813 non-null  object 
 3   Number.of.Engines       28813 non-null  float64
 4   Total.Fatal.Injuries    28813 non-null  float64
 5   Total.Serious.Injuries  28813 non-null  float64
 6   Total.Minor.Injuries    28813 non-null  float64
 7   Total.Uninjured         28813 non-null  float64
dtypes: float64(5), object(3)
memory usage: 2.0+ MB


In [220]:
ad.head()

,Accident.Number,Aircraft.Category,Amateur.Built,Number.of.Engines,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured
5,NYC79AA106,Airplane,No,2.0,0.0,0.0,1.0,44.0
7,SEA82DA022,Airplane,No,1.0,0.0,0.0,0.0,2.0
8,NYC82DA015,Airplane,No,2.0,0.0,0.0,0.0,2.0
12,FTW82FRJ07,Airplane,No,1.0,0.0,0.0,1.0,0.0
13,FTW82FRA14,Airplane,No,1.0,1.0,0.0,0.0,0.0


In [221]:
ad['Aircraft.Category'].describe()

count        28813
unique          14
top       Airplane
freq         24848
Name: Aircraft.Category, dtype: object

In [222]:
ad['Aircraft.Category'].unique()

array(['Airplane', 'Helicopter', 'Glider', 'Balloon', 'Gyrocraft',
       'Ultralight', 'Unknown', 'Blimp', 'Powered-Lift', 'Weight-Shift',
       'Powered Parachute', 'Rocket', 'WSFT', 'ULTR'], dtype=object)

In [223]:
ad['Amateur.Built'].describe()

count     28813
unique        2
top          No
freq      25479
Name: Amateur.Built, dtype: object

In [224]:
ad['Amateur.Built'].unique()

array(['No', 'Yes'], dtype=object)

# Data Preparation

## Data Cleaning

In [212]:
"""
Here I'm going to drop all of the columns that aren't of interest to me.  I'm interested in 
knowing whehter amateur vs. professional built aircrafts are lower risk, which aircraft type
is lowest risk, and which number of engines are lowest risk based on the number of injuries
they have.
"""

ad.drop(columns = ['Event.Id', 'Investigation.Type', 'Event.Date', 'Location', 'Country', 'Latitude', 
                  'Airport.Code', 'Airport.Name', 'Aircraft.damage', 'Registration.Number', 'FAR.Description', 
                  'Schedule', 'Purpose.of.flight', 'Air.carrier', 'Weather.Condition', 'Longitude', 
                  'Broad.phase.of.flight', 'Report.Status', 'Publication.Date', 'Injury.Severity', 'Make',
                  'Model', 'Engine.Type'], inplace=True)

In [214]:
ad = ad.drop_duplicates() #drop duplicate values

In [215]:
### Now I have all of the columns I want to do my analysis so I'm going to check and see if those
### columns have any missing data. 

ad.isna().sum()

Accident.Number               1
Aircraft.Category         56601
Amateur.Built               103
Number.of.Engines          6077
Total.Fatal.Injuries      11402
Total.Serious.Injuries    12511
Total.Minor.Injuries      11934
Total.Uninjured            5913
dtype: int64

In [216]:
""" The injuries seem to have a large amount of missing data but it looks like if there were no
injuries that they left the cell blank so I'm going to replace all of these missing data with 0's"""

ad['Total.Fatal.Injuries'] = ad['Total.Fatal.Injuries'].fillna(value=0)
ad['Total.Serious.Injuries'] = ad['Total.Serious.Injuries'].fillna(value=0)
ad['Total.Minor.Injuries'] = ad['Total.Minor.Injuries'].fillna(value=0)
ad['Total.Uninjured'] = ad['Total.Uninjured'].fillna(value=0)

In [217]:
### the rest of the missing data makes up about 8% of a large dataset so I'm going to remove those
ad = ad.dropna()

In [218]:
###Now I have finished cleaning the Data and I have 8 columns with 28k accidends of information that I'm interested in 

print('Sum of Missing Values in Each Column:  ', ad.isna().sum())
print('Shape of the dataset:  ', ad.shape)

Sum of Missing Values in Each Column:   Accident.Number           0
Aircraft.Category         0
Amateur.Built             0
Number.of.Engines         0
Total.Fatal.Injuries      0
Total.Serious.Injuries    0
Total.Minor.Injuries      0
Total.Uninjured           0
dtype: int64
Shape of the dataset:   (28813, 8)


# Analysis

In [207]:
### The first thing I wanted to do was see if there was a difference in the amount of people 
### injured in aircraft that were built by amateurs or not. 

ad.groupby(['Amateur.Built']).sum()

,Number.of.Engines,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured
Amateur.Built,,,,,
No,29182.0,10535.0,6733.0,5366.0,124152.0
Yes,3351.0,979.0,748.0,724.0,2142.0


In [183]:
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

### Create plot
amateur_built_fig, injuries_axes = plt.subplots(nrows=1, ncols=2, figsize=(10,4))

injuries_axes[0].set_title('Injuries in Amateur vs. Professional Built Aircraft')
injuries_axes[0].set_ylabel('Number of Injuries')
injuries_axes[0].set_xlabel('Amateur vs. Professional')

injuries = ad['Total.Fatal.Injuries']

injuries_axes[0].hist(injuries, bins =15)

plt.show()

In [147]:
### Now we are going to drop all of events that happened in amateur-built airplanes because
### I'm not going to make a recommendation to my company to buy amateur-built airplanes. 

ad = ad[ad['Amateur.Built'] == 'No']
        

In [148]:
#Now I dropped the Amateur Built Column because all of the data is non-amateur-built airplanes. 

ad = ad.drop('Amateur.Built', axis = 1)

In [149]:
### Now I have all of the columns I want to do my analysis so I'm going to check and see if those
### columns have any missing data. 

ad.isna().sum()

Accident.Number               0
Aircraft.Category         51591
Number.of.Engines          5706
Engine.Type                6664
Total.Fatal.Injuries      10157
Total.Serious.Injuries    11084
Total.Minor.Injuries      10557
Total.Uninjured            4887
dtype: int64

In [180]:
""" The injuries seem to have a large amount of missing data but it looks like if there were no
injuries that they left the cell blank so I'm going to replace all of these missing data with 0's"""

ad['Total.Fatal.Injuries'] = ad['Total.Fatal.Injuries'].fillna(value=0)
ad['Total.Serious.Injuries'] = ad['Total.Serious.Injuries'].fillna(value=0)
ad['Total.Minor.Injuries'] = ad['Total.Minor.Injuries'].fillna(value=0)
ad['Total.Uninjured'] = ad['Total.Uninjured'].fillna(value=0)

In [181]:
### the rest of the missing data makes up about 8% of a large dataset so I'm going to remove those
ad = ad.dropna()


In [182]:
###Now I have finished cleaning the Data and I have 10 columns with 22k accidends of information that I'm interested in 

print('Sum of Missing Values in Each Column:  ', ad.isna().sum())
print('Shape of the dataset:  ', ad.shape)

Sum of Missing Values in Each Column:   Accident.Number           0
Aircraft.Category         0
Amateur.Built             0
Number.of.Engines         0
Engine.Type               0
Total.Fatal.Injuries      0
Total.Serious.Injuries    0
Total.Minor.Injuries      0
Total.Uninjured           0
dtype: int64
Shape of the dataset:   (25953, 9)


In [153]:
""" First, I'm charged with determining which aircraft are the lowest risk for the company. 
Therefore, I grouped the data by aircraft category to see which of the aircraft types has 
the lowest risk"""

###TODO: Calculate the percentage of passengers who aren't injured by the accident to claim which type of aircraft is the safest

ad.groupby(['Aircraft.Category']).count()


,Accident.Number,Number.of.Engines,Engine.Type,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured
Aircraft.Category,,,,,,,
Airplane,19910,19910,19910,19910,19910,19910,19910
Balloon,45,45,45,45,45,45,45
Blimp,4,4,4,4,4,4,4
Glider,115,115,115,115,115,115,115
Gyrocraft,25,25,25,25,25,25,25
Helicopter,2523,2523,2523,2523,2523,2523,2523
Powered Parachute,78,78,78,78,78,78,78
Powered-Lift,2,2,2,2,2,2,2
Rocket,1,1,1,1,1,1,1


In [154]:
### Airplanes have the lowest risk so I'm going to just look at those. 

ad = ad[ad['Aircraft.Category'] == 'Airplane']

In [155]:
### I don't need the Aircraft Category anymore because they're all planes

ad = ad.drop('Aircraft.Category', axis = 1)

In [156]:
###Next I want to see if there's a difference in the percentage of uninjured people involved in
### airplane accidents based on the number of engines the plane has. 

### Next I checked to see how many accidents I have from each accident type.  

ad.groupby(['Number.of.Engines']).count()


,Accident.Number,Engine.Type,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured
Number.of.Engines,,,,,,
0.0,7,7,7,7,7,7
1.0,16838,16838,16838,16838,16838,16838
2.0,2943,2943,2943,2943,2943,2943
3.0,47,47,47,47,47,47
4.0,74,74,74,74,74,74
8.0,1,1,1,1,1,1


In [157]:
### I'm going to remove the accidents from 0 and 8 engines because I don't have enough data from those.

ad = ad[ad['Number.of.Engines'] != 0.00]
ad = ad[ad['Number.of.Engines'] != 8.0]

In [168]:
ad.groupby(['Number.of.Engines']).mean()

,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Percent.Uninjured
Number.of.Engines,,,,,
3.0,0.085106,0.553191,2.042553,106.978723,0.953007


In [165]:
ad['Percent.Uninjured'] = ad['Total.Uninjured']/(ad['Total.Fatal.Injuries']+ad['Total.Serious.Injuries']+ad['Total.Minor.Injuries']+ad['Total.Uninjured'])


,Number.of.Engines,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured
Percent.Uninjured,,,,,
0.000000,3.0,2.0,0.0,2.0,0.0
0.750000,3.0,0.0,0.0,1.0,3.0
0.800000,3.0,0.0,1.0,0.0,4.0
0.818841,3.0,0.0,0.0,25.0,113.0
0.845238,3.0,0.0,7.0,19.0,142.0
0.858491,3.0,2.0,4.0,24.0,182.0
0.933333,3.0,0.0,11.0,1.0,168.0
0.935484,3.0,0.0,0.0,14.0,203.0
0.965753,3.0,0.0,1.0,9.0,282.0


In [160]:
ad = ad[ad['Number.of.Engines'] == 3.0]

In [167]:
ad.groupby(['Engine.Type']).mean()

,Number.of.Engines,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Percent.Uninjured
Engine.Type,,,,,,
Turbo Fan,3.0,0.088889,0.577778,2.133333,109.777778,0.950821
Turbo Jet,3.0,0.000000,0.000000,0.000000,44.000000,1.000000


In [23]:
ad[ad.duplicated(keep=False)].sort_values(by='Investigation.Type')

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
85351,NaN,01-06-2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86122,NaN,01-06-2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86140,NaN,01-06-2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86119,NaN,01-06-2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86154,NaN,01-06-2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86349,NaN,31-08-2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
85193,NaN,31-08-2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86717,NaN,31-08-2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86561,NaN,31-08-2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
ad.isna().sum() #checked for missing data

Event.Id                   1459
Investigation.Type            0
Accident.Number            1459
Event.Date                 1459
Location                   1511
Country                    1685
Latitude                  55966
Longitude                 55975
Airport.Code              40099
Airport.Name              37558
Injury.Severity            2459
Aircraft.damage            4653
Aircraft.Category         58061
Registration.Number        2776
Make                       1522
Model                      1551
Amateur.Built              1561
Number.of.Engines          7543
Engine.Type                8536
FAR.Description           58325
Schedule                  77766
Purpose.of.flight          7651
Air.carrier               73700
Total.Fatal.Injuries      12860
Total.Serious.Injuries    13969
Total.Minor.Injuries      13392
Total.Uninjured            7371
Weather.Condition          5951
Broad.phase.of.flight     28624
Report.Status              7840
Publication.Date          16689
dtype: i

In [17]:
ad['Make'].value_counts()

Cessna                           22227
Piper                            12029
CESSNA                            4922
Beech                             4330
PIPER                             2841
                                 ...  
William F. Lair                      1
AIRGYRO                              1
Quad City Ultralight                 1
QUARTZ MOUNTAIN AEROSPACE INC        1
Harold Vander                        1
Name: Make, Length: 8237, dtype: int64

In [18]:
ad['Model'].value_counts()


152                     2367
172                     1756
172N                    1164
PA-28-140                932
150                      829
                        ... 
KCA                        1
HOFFMAN BMW AIRTRIKE       1
P-2                        1
300A                       1
MIRAGE F1 B                1
Name: Model, Length: 12318, dtype: int64

Exploratory Data Analysis

Conclusions

Limitations

Recommendations

Next Steps